# From the sudoku solver adjusted for
# SUDOKU MIX  
## Tubantia sudoku mix from 5 june upper right corner used plus rest upper row
At several places on the www to be found, e.g. below in https://ericpony.github.io/z3py-tutorial/guide-examples.htm
<img src="sudokumix5jun.jpg"/>

In [1]:
from mydateFP import getFPnameDate

2019-06-10 11:54:08.185034


In [2]:
!cat mydateFP.py

def getFPnameDate(voor = "mixSodudoku"):
    from datetime  import datetime
    d0, d1  = str(datetime.now()).split(" ")
    print (d0,d1)
    return voor + (d0[5:] + "_" + d1[0:2] + d1[3:5]).replace("-","") + ".py"
getFPnameDate()


# a Sudoku MIX 3x9

In [3]:
# show at most 41 solutions
maxaantal = 41
from z3 import *
from mydateFP import getFPnameDate

#print inbetween results
tussenresultaat = True
fpname = getFPnameDate()
print("results will be saved in  (this working dir): ", fpname)
fp = open(fpname , "w")

# a 3x3 Sudoku where ONE number is given a ^ lower right corner
nulrij = [0 for i in range(9)]
NulSodoku = [ nulrij for i in range(3)]

NulSodoku[0] = [0,0,0,0,0,0,0,0,0]
NulSodoku[1] = [0,0,0,0,0,0,0,0,0]
NulSodoku[2] = [0,0,0,0,0,0,0,0,6]

instance = NulSodoku

"""
# show at most 41 solutions
maxaantal = 40
from z3 import *
from mydateFP import getFPnameDate

#print inbetween results
tussenresultaat = True
fpname = getFPnameDate()
print("results will be saved in  (this working dir): ", fpname)
fp = open(fpname , "w")
"""
# a 3x3 Sudoku where ONE number is given a ^ lower right corner
nulrij = [0 for i in range(9)]
NulSodoku = [ nulrij for i in range(3)]

NulSodoku[0] = [0,0,0,0,0,0,0,0,0]
NulSodoku[1] = [0,0,0,0,0,0,0,0,0]
NulSodoku[2] = [0,0,0,0,0,0,0,0,6]

instance = NulSodoku

# 3x9 matrix of integer variables, zero based indexing!
X = [ [ Int("x_%s_%s" % (i, j)) for j in range(9) ]
      for i in range(3) ]

# each cell contains a value in {1, ..., 9}
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
             for i in range(3) for j in range(9) ]
#PKHG>tothier
# each row contains a digit at most once
rows_c   = [ Distinct(X[i]) for i in range(3) ]

# each column contains a digit at most once
cols_c   = [ Distinct([ X[i][j] for i in range(3)]) for j in range(9)]
 

# each 3x3 square contains a digit at most once
"""
sq_c     = [ Distinct([ X[i0 + i][j0 + j ]
                        for i in range(3) for j in range(3) ])
             for i0 in range(1) for j0 in range(3) ]
"""
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                        for i in range(3) for j in range(3) ])
             for i0 in range(1) for j0 in range(3) ]


    

# rectangles in the 3x9 Sudoku are given:
c00_10 = [And(X[0][6] + X[0][7]  == 9),
          And(X[1][6] + X[1][7]  == 5),
          And(X[2][6] + X[2][7] == 13),
        #subtract the known 6 in last column gives:
          And(X[0][8] + X[1][8] == 18 - 6),
          And(X[2][8] == 6), #up to here seems ok
          
          And(X[0][0] + X[1][0] == 11),
          And(X[0][1] + X[1][1] == 9),
          And(X[0][2] + X[0][3] == 14),
          And(X[1][2] + X[2][2] == 9),
          
          And(X[0][4] + X[0][5] == 9),
          And(X[1][3] + X[2][3] == 9),
          And(X[1][4] + X[2][4] == 5),
          #PKHG>??? And(X[1][5] + X[2][5] == 13)
         ]
         #And(X[][] + X[][] == ),
         
         
#all conditions put together
sudoku_c = cells_c + rows_c + cols_c + sq_c + c00_10

# sudoku instance, we use '0' for empty cells

instance_c = [ If(instance[i][j] == 0,
                  True,
                  X[i][j] == instance[i][j])
               for i in range(3) for j in range(3) ]
allSolutions = []
s = Solver()
s.add(sudoku_c + instance_c) 

#while loop will stop if counter is > maxaantal, or 
#no new solutions anymore found
counter  = 0
while s.check() == sat:
    counter += 1
    fp.write("p" + str(counter) + " = ")
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(3) ]
    t = ""
    for el in r:
    #print(el)
        res = [ "" + str(c) for c in el]
        tmp = "".join(res)
        t += tmp
    fp.write(t + "\n")
    
    ns = int(t)
    allSolutions.append(ns)
    
    #print(len(allSolutions))
    #print(counter , t)
    if tussenresultaat:
        print_matrix(r)
    
    # the solution found will be not anymore allowed: Not(And(vals))
    vals = [X[i][j] == m.evaluate(X[i][j]) for j in range(3)
            for i in range(3)]
    s.add(Not(And(vals)))
    if counter > maxaantal:
        break
fp.close()
allSolasSet = set(allSolutions)
if counter:
    print("all different?" , len(allSolutions) == counter)
else:
    print("no solution, check instance", instance)
                    

2019-06-10 11:54:13.913621
results will be saved in  (this working dir):  mixSodudoku0610_1154.py
[[4, 6, 5, 9, 8, 1, 2, 7, 3],
 [7, 3, 8, 5, 2, 6, 1, 4, 9],
 [9, 2, 1, 4, 3, 7, 5, 8, 6]]
[[4, 6, 5, 9, 1, 8, 2, 7, 3],
 [7, 3, 8, 5, 2, 6, 4, 1, 9],
 [2, 9, 1, 4, 3, 7, 5, 8, 6]]
all different? True


In [4]:
# last 3x3 cotains no 1 and 4???  ==> errro

In [5]:
counter

2

# filename from an earlier run!
## change if you like

In [18]:
fpname

'mixSodudoku0610_1154.py'

In [19]:
!cat mixSodudoku0610_1154.py

p1 = 465981273738526149921437586
p2 = 465918273738526419291437586


In [20]:
from mixSodudoku0610_1154 import *

In [21]:
plv = [eval(el) for el in str(p1)]
print(plv)

[4, 6, 5, 9, 8, 1, 2, 7, 3, 7, 3, 8, 5, 2, 6, 1, 4, 9, 9, 2, 1, 4, 3, 7, 5, 8, 6]


In [62]:
import numpy as np
def convtomat(pw, mr = 3, mc = 9):
    
    tmp = np.array([eval(el) for el in list(str(pw))])
    return np.matrix(tmp).reshape(mr,mc)
p2_np = convtomat(p2)
p2_np

matrix([[4, 6, 5, 9, 1, 8, 2, 7, 3],
        [7, 3, 8, 5, 2, 6, 4, 1, 9],
        [2, 9, 1, 4, 3, 7, 5, 8, 6]])

In [78]:
allv = np.array([eval(el) for el in list(str(p2))])
allv.shape

(27,)

In [82]:
allv2 = allv.reshape((3,9))
print(allv2)

[[4 6 5 9 1 8 2 7 3]
 [7 3 8 5 2 6 4 1 9]
 [2 9 1 4 3 7 5 8 6]]


# pandas is sometimes a bit nicer then numpy

In [23]:
import pandas as pd

In [24]:
pdm = pd.DataFrame(p2_np,columns=['a','b','c','d','e','f','g','h','i'])
pdm

,a,b,c,d,e,f,g,h,i
0,4,6,5,9,1,8,2,7,3
1,7,3,8,5,2,6,4,1,9
2,2,9,1,4,3,7,5,8,6


In [35]:
sum(pdm['a']) +sum(pdm['b']) + sum(pdm['c'])

45

In [28]:
pdm2 = pd.DataFrame(p2_np,columns=[i for i in range(9)])
pdm2

,0,1,2,3,4,5,6,7,8
0,4,6,5,9,1,8,2,7,3
1,7,3,8,5,2,6,4,1,9
2,2,9,1,4,3,7,5,8,6


In [32]:
[sum(pdm2.T[i]) for i in range(3)]

[45, 45, 45]

In [52]:
#sum of the 3x3 parts
[sum(pdm2[0 + 3 * i]) + sum(pdm2[1 + 3 * i])+ sum(pdm2[2 + 3 * i]) for i in range(3)]

[45, 45, 45]

# check (in this case) if the last colums
# sum is 18

In [56]:
pdm['i'] == pdm2[8]

0    True
1    True
2    True
dtype: bool

In [58]:
sum(pdm2[8])

18

# because row conditions to sum
matrix has to be transposed (easy: .T )

In [13]:
sum(pdm.T[2])

45

# check sum of the first two cels sums to 9
(though easily seen from the matrix print above)

In [14]:
pdm['a'][0] + pdm['b'][0] == 9

False

In [17]:
type(pdm)

pandas.core.frame.DataFrame